In [1]:
from datasets import load_dataset
import pandas as pd

train_dataset = load_dataset("swan07/authorship-verification", split="train[:80000]")
test_dataset = load_dataset("swan07/authorship-verification", split="test[:16000]")

from specter import SpecterVectorizer

spectorVectorizer = SpecterVectorizer()
spectorVectorizer.load()

train_spector = spectorVectorizer.embed(train_dataset)
test_spector = spectorVectorizer.embed(test_dataset)

from bert import BertVectorizer

bertVectorizer = BertVectorizer()
bertVectorizer.load()

train_bert = bertVectorizer.embed(train_dataset)
test_bert = bertVectorizer.embed(test_dataset)



c:\Users\mamy0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 16000/16000 [02:06<00:00, 126.00it/s]


In [2]:
from pathlib import Path
import pickle

Path("./embeddings").mkdir(exist_ok=True)

# 저장
print("저장 중...")
with open('./embeddings/train_specter.pkl', 'wb') as f:
    pickle.dump(train_spector, f)
print("✓ train_specter.pkl")

with open('./embeddings/test_specter.pkl', 'wb') as f:
    pickle.dump(test_spector, f)
print("✓ test_specter.pkl")

with open('./embeddings/train_bert.pkl', 'wb') as f:
    pickle.dump(train_bert, f)
print("✓ train_bert.pkl")

with open('./embeddings/test_bert.pkl', 'wb') as f:
    pickle.dump(test_bert, f)
print("✓ test_bert.pkl")


저장 중...
✓ train_specter.pkl
✓ test_specter.pkl
✓ train_bert.pkl
✓ test_bert.pkl


In [1]:
from pathlib import Path
import pickle

with open('./embeddings/train_bert.pkl', 'rb') as f:
    bert_train_data = pickle.load(f)
with open('./embeddings/test_bert.pkl', 'rb') as f:
    bert_test_data = pickle.load(f)

with open('./embeddings/train_bert.pkl', 'rb') as f:
    spector_train_data = pickle.load(f)
with open('./embeddings/test_bert.pkl', 'rb') as f:
    spector_test_data = pickle.load(f)

In [2]:
from classifier import train_authorship_classifiers
models, results_bert = train_authorship_classifiers(bert_train_data, bert_test_data, True)
models, results_spector = train_authorship_classifiers(spector_train_data, spector_test_data, True)

Using Device: cuda
Preparing features on GPU…
=== Training LightGBM (GPU) ===
[LightGBM] [Info] Number of positive: 45491, number of negative: 34509
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 979455
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 3841
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 3841 dense feature groups (293.27 MB) transferred to GPU in 0.102038 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.568638 -> initscore=0.276294
[LightGBM] [Info] Start training from score 0.276294
=== Training XGBoost (GPU) ===
=== Training CatBoost (GPU) ===


=== Training Neural Network (GPU) ===
Epoch 1/10 - Loss 0.4867 - TestAcc 0.7271
Epoch 2/10 - Loss 0.4349 - TestAcc 0.7348
Epoch 3/10 - Loss 0.4143 - TestAcc 0.7378
Epoch 4/10 - Loss 0.3910 - TestAcc 0.7409
Epoch 5/10 - Loss 0.3699 - TestAcc 0.7449
Epoch 6/10 - Loss 0.3504 - TestAcc 0.7450
Epoch 7/10 - Loss 0.3275 - TestAcc 0.7426
Epoch 8/10 - Loss 0.3093 - TestAcc 0.7452
Epoch 9/10 - Loss 0.2858 - TestAcc 0.7411
Epoch 10/10 - Loss 0.2682 - TestAcc 0.7405
Using Device: cuda
Preparing features on GPU…
=== Training LightGBM (GPU) ===
[LightGBM] [Info] Number of positive: 45491, number of negative: 34509
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 979455
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 3841
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size 

=== Training Neural Network (GPU) ===
Epoch 1/10 - Loss 0.4859 - TestAcc 0.7243
Epoch 2/10 - Loss 0.4345 - TestAcc 0.7331
Epoch 3/10 - Loss 0.4144 - TestAcc 0.7378
Epoch 4/10 - Loss 0.3918 - TestAcc 0.7401
Epoch 5/10 - Loss 0.3710 - TestAcc 0.7452
Epoch 6/10 - Loss 0.3489 - TestAcc 0.7370
Epoch 7/10 - Loss 0.3280 - TestAcc 0.7396
Epoch 8/10 - Loss 0.3080 - TestAcc 0.7360
Epoch 9/10 - Loss 0.2917 - TestAcc 0.7426
Epoch 10/10 - Loss 0.2681 - TestAcc 0.7392


In [13]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


class MajorityVotingEnsemble:
    def __init__(self, model_names):
        self.model_names = model_names  # 예: ['pytorch_nn', 'xgboost', 'catboost']

    def predict(self, df_pred):
        pred_cols = [f"pred_{m}" for m in self.model_names]
        return df_pred[pred_cols].mode(axis=1)[0]

    def compute_disagreement_rate(self, df_pred):
        pred_cols = [f"pred_{m}" for m in self.model_names]
        preds = df_pred[pred_cols].values

        disagreement_rates = []
        for row in preds:
            values, counts = np.unique(row, return_counts=True)
            max_count = counts.max()
            total = len(row)
            disagreement = 1 - (max_count / total)
            disagreement_rates.append(disagreement)

        return np.mean(disagreement_rates)

    def evaluate(self, df_pred, true_label_col="true_label"):
        y_true = df_pred[true_label_col]
        y_pred = self.predict(df_pred)

        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        disagreement_rate = self.compute_disagreement_rate(df_pred)

        print("\n==============================")
        print("🔮 Majority Voting Ensemble 결과")
        print("==============================")
        print(f"Accuracy: {acc:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Disagreement Rate: {disagreement_rate:.4f}")
        print("==============================\n")

        return {
            "accuracy": acc,
            "f1": f1,
            "disagreement_rate": disagreement_rate
        }

In [14]:
import pandas as pd

majority_voting = MajorityVotingEnsemble(models.keys())
df_pred = pd.DataFrame({
    "true_label": bert_test_data["same"],  # 정답 레이블
    "pred_lightgbm": results_bert.iloc[0]["pred_test"],
    "pred_xgboost": results_bert.iloc[1]["pred_test"],
    "pred_catboost": results_bert.iloc[2]["pred_test"],
    "pred_simple_nn": results_bert.iloc[3]["pred_test"],
})
majority_voting.evaluate(df_pred)


🔮 Majority Voting Ensemble 결과
Accuracy: 0.7348
F1 Score: 0.7260
Disagreement Rate: 0.0849



{'accuracy': 0.7348125,
 'f1': 0.7259575017761416,
 'disagreement_rate': np.float64(0.084890625)}

In [15]:
import pandas as pd

majority_voting = MajorityVotingEnsemble(models.keys())
df_pred = pd.DataFrame({
    "true_label": spector_test_data["same"],  # 정답 레이블
    "pred_lightgbm": results_spector.iloc[0]["pred_test"],
    "pred_xgboost": results_spector.iloc[1]["pred_test"],
    "pred_catboost": results_spector.iloc[2]["pred_test"],
    "pred_simple_nn": results_spector.iloc[3]["pred_test"],
})
majority_voting.evaluate(df_pred)


🔮 Majority Voting Ensemble 결과
Accuracy: 0.7352
F1 Score: 0.7276
Disagreement Rate: 0.0885



{'accuracy': 0.7351875,
 'f1': 0.7276467185189947,
 'disagreement_rate': np.float64(0.08846875)}

In [17]:
import pandas as pd

majority_voting = MajorityVotingEnsemble(["simple_nn_spector", "simple_nn_bert"])
df_pred = pd.DataFrame({
    "true_label": spector_test_data["same"],  # 정답 레이블
    "pred_simple_nn_spector": results_spector.iloc[3]["pred_test"],
    "pred_simple_nn_bert": results_bert.iloc[3]["pred_test"],
})
majority_voting.evaluate(df_pred)


🔮 Majority Voting Ensemble 결과
Accuracy: 0.7400
F1 Score: 0.7233
Disagreement Rate: 0.0705



{'accuracy': 0.74,
 'f1': 0.7232570516232039,
 'disagreement_rate': np.float64(0.07053125)}